In [ ]:
# ===== Pacotes / Packages =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
# ===== 1. Ler os dados / Read the data .XLSX =====
dados = pd.read_excel("NAME.xlsx")

In [ ]:
# ===== 1. Ler os dados / Read the data .CSV =====
dados = pd.read_csv("NAME.csv")

In [ ]:
# A classe deve estar na primeira coluna / The class must be in the first column
dados["Classe"] = dados.iloc[:, 0].astype(str)  # converter para string/categoria / convert to string/category
dados = dados.drop(dados.columns[0], axis=1)    # remover a coluna original / remove the original column
cols = list(dados.columns)
cols = cols[:-1] + ["Classe"]                   # garantir que "Classe" fique por último / ensure that "Class" comes last
dados = dados[cols]

In [ ]:
# ===== 2. Garantir que preditoras sejam numéricas / Ensure predictors are numeric =====
dados_pred = dados.drop("Classe", axis=1).apply(pd.to_numeric, errors="coerce")

# Remover colunas constantes / Remove constant columns
constantes = [c for c in dados_pred.columns if dados_pred[c].nunique() == 1]
dados_pred = dados_pred.drop(columns=constantes)

# Remover colunas com NA / Remove columns with NA
dados_pred = dados_pred.dropna(axis=1)

# Recombinar com a classe / Recombine with the class
dados = pd.concat([dados_pred, dados["Classe"]], axis=1)

In [ ]:
# ===== 3. Divisão treino/teste / Training/testing division =====
X = dados.drop("Classe", axis=1).values
y = dados["Classe"].values

# Codificar classes como números / Encode classes as numbers
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234, stratify=y
)

In [ ]:
# ===== 4. Rodar QDA / Run QDA =====
modelo_qda = QuadraticDiscriminantAnalysis()
modelo_qda.fit(X_train, y_train)

In [ ]:
# ===== 5. Predição / Prediction =====
pred_treino = modelo_qda.predict(X_train)
pred_teste = modelo_qda.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, pred_treino))
print("Test Accuracy:", accuracy_score(y_test, pred_teste))

In [ ]:
# ===== 6. Matriz de Confusão / Confusion Matrix =====
y_test_int = y_test.astype(int)
pred_teste_int = pred_teste.astype(int)

cm = confusion_matrix(y_test_int, pred_teste_int, labels=np.unique(y_test_int))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix -Test Set")
plt.show()

# Relatório detalhado / Detailed report (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_test, pred_teste, target_names=le.classes_))
